    16_230110_114_agg_combine_map1
        
        data : /aiffel/000_aiffelthon_230111/3김연수/002_data_dir/001_map_data/kyungki_region.json
        /aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/20210823_socar_zone.csv
        /aiffel/000_aiffelthon_230111/3김연수/002_data_dir/003_search_data/happy_house.csv
        /aiffel/000_aiffelthon_230111/3김연수/002_data_dir/003_search_data/k_university.csv

In [80]:
#1. package load
import requests
import json
import os
import math
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

import warnings
warnings.filterwarnings(action='ignore') 

print(pd.__version__)
print(gpd.__version__)
print(folium.__version__)

1.4.4
0.12.2
0.14.0


In [81]:
#2. 프로젝트(1)  ... data load

path = 'c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/004_node_data/20210823_socar_zone.csv'

socar_zone_info_df = pd.read_csv(path)
socar_zone_info_df.head()

,zone_name,lng,lat,region1,region2,region3
0,비전2 파출소사거리,127.102939,36.995989,경기도,평택시,비전동
1,수원시청역 9번출구,127.032953,37.262378,경기도,수원시 팔달구,인계동
2,목포 상동현대아파트 앞,126.415655,34.811289,전라남도,목포시,상동
3,단국대 산학협력관 앞,127.166626,36.832965,충청남도,천안시 동남구,신부동
4,목원대학교 앞,127.337858,36.329906,대전광역시,서구,도안동


In [82]:
k_socar_zone_df = socar_zone_info_df[socar_zone_info_df['region1'] == '경기도']

In [83]:
#3.  행복주택 load
path = 'c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/happy_house_df1.csv'
happy_house_df = pd.read_csv(path)
happy_house_df.head()                       

,지역,단지,세대수,주소,lng,lat
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,126.865491,37.650992
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,126.921037,37.653331
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,126.914946,37.654038
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,126.868888,37.652631
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,126.847648,37.621215


In [84]:
#3.  대학교 load
path = 'c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/k_university.csv'
k_university_df = pd.read_csv(path)
k_university_df.head()

,학교구분,학교코드,학교명,본분교,학제,지역,설립구분,관련법령,법인명,학교상태,...,우편번호,학교개교일,학교홈페이지,총장(대학원장)명,학교대표\r\n번호,학교대표\r\n팩스번호,x,y,lng,lat
0,대학,37.0,한경대학교,본교,대학교,경기,국립,고등교육법,해당없음,기존,...,17579,2012-03-01,www.hknu.ac.kr,이원희,031-670-5114,031-670-5469,978801.712957,1.890391e+06,127.261716,37.011780
1,대학,46.0,가톨릭대학교,본교,대학교,경기,사립,고등교육법,가톨릭학원,기존,...,14662,1995-03-01,www.catholic.ac.kr,원종철,02-2164-4114,02-2164-4754,938317.711386,1.943204e+06,126.802293,37.486022
2,대학,51.0,강남대학교,본교,대학교,경기,사립,고등교육법,강남학원,기존,...,16979,1948-08-12,www.kangnam.ac.kr,윤신일,031-280-3500,031-280-3173,967379.084547,1.919514e+06,127.132051,37.273974
3,대학,56.0,경기대학교,본교,대학교,경기,사립,고등교육법,경기학원,기존,...,16227,1947-11-08,www.kyonggi.ac.kr,전준철(직무대행),031-249-9114,031-243-2194,958688.169183,1.922509e+06,127.033857,37.300618
4,대학,63.0,가천대학교,본교,대학교,경기,사립,고등교육법,가천학원,기존,...,13120,2012-03-01,www.gachon.ac.kr,이길여,031-750-5114,031-753-8828,967163.380457,1.939303e+06,127.128740,37.452335


In [85]:
#3. university 위치데이터 리스트 만들기
k_university_location_data = [[row["lat"], row["lng"]] for _, row in k_university_df.iterrows()]

In [86]:
#3. 지하철 load 

path = 'c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/001_map_view_real/02_map_subway/subway_total_3.csv'
k_subway_info_df = pd.read_csv(path)
k_subway_info_df.head()


,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,코레일,1호선,가능,경기도 의정부시 가능동 197-1,경기도 의정부시 평화로 633,959850.319275,1.972172e+06,127.044255,37.748305
1,코레일,1호선,가산디지털단지,서울특별시 금천구 가산동 468-4,서울특별시 금천구 벚꽃로 309,945373.187085,1.942741e+06,126.882129,37.482290
2,코레일,1호선,간석,인천광역시 남동구 간석4동 762번지,인천광역시 남동구 석정로 522-14,928718.830404,1.940941e+06,126.693945,37.464929
3,코레일,1호선,개봉,서울특별시 구로구 개보동 415,서울특별시 구로구 경인로40길 47,943348.199060,1.944111e+06,126.859121,37.494522
4,코레일,1호선,관악,경기도 안양시 만악구 석수동 101-16,경기도 안양시 만안구 경수대로1273번길 46,947714.862265,1.935684e+06,126.909115,37.418821


In [87]:
#3. 지하철 위치데이터 리스트 만들기
k_subway_location_data = [[row["lat"], row["lng"]] for _, row in k_subway_info_df.iterrows()]

In [88]:
#3. 아파트 load   
path = 'c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/001_map_view_real/01_map_apt/k_apt_df.csv'
k_apt_df = pd.read_csv(path)
k_apt_df.head()

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
0,안양시,DY town,호계3동,971-1,2013-01-03,9,2,115,N,경기도 안양시 호계3동 971-1,951780.982196,1.930038e+06,126.955433,37.368156
1,안양시,HHI브라운빌3차아파트,호계2동,940,2007-07-27,15,2,149,N,경기도 안양시 호계2동 940,951648.800973,1.930982e+06,126.953879,37.376652
2,안양시,LD범계마을아파트,호계2동,923-30,1999-01-13,17,1,113,N,경기도 안양시 호계2동 923-30,951221.572581,1.931673e+06,126.949008,37.382859
3,안양시,LG빌리지,석수2동,484,2001-05-19,10,34,1872,Y,경기도 안양시 석수2동 484,947025.809511,1.936663e+06,126.901258,37.427608
4,안양시,강남6차아파트,비산2동,566-5,1984-10-25,6,1,54,N,경기도 안양시 비산2동 566-5,950089.066159,1.933251e+06,126.936109,37.397025


In [89]:
k_apt_df[k_apt_df['lng'].isnull()]

,시군명,공동주택명정보,읍면동주소,지번주소,사용검사일자,층수,동수,세대수,의무관리대상여부,addr,x,y,lng,lat
43,안양시,다청림카운티,박달1동,149-12,2016-06-13,5,10,104,N,경기도 안양시 박달1동 149-12,NaN,NaN,NaN,NaN
50,안양시,대흥아파트,박달1동,13241,1987-07-09,5,1,50,N,경기도 안양시 박달1동 13241,NaN,NaN,NaN,NaN
61,안양시,동원베네스트아파트,박달1동,44900,2007-07-13,15,2,134,N,경기도 안양시 박달1동 44900,NaN,NaN,NaN,NaN
67,안양시,리치밸리주상복합,호계1동,966-1,2006-08-09,19,1,70,N,경기도 안양시 호계1동 966-1,NaN,NaN,NaN,NaN
93,안양시,미주아파트,안양6동,423-3,1984-06-07,5,1,60,N,경기도 안양시 안양6동 423-3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6907,용인시,풍산햇빛마을아파트,처인구모현읍,464-2,2000-03-24,22,4,330,Y,경기도 용인시 처인구모현읍 464-2,NaN,NaN,NaN,NaN
6925,용인시,한샘 더랜드마크,수지구풍덕천동,30195,2018-11-28,19,1,220,Y,경기도 용인시 수지구풍덕천동 30195,NaN,NaN,NaN,NaN
6926,용인시,한천마을 금광베네스타,처인구이동읍,1134,2008-07-30,15,9,460,Y,경기도 용인시 처인구이동읍 1134,NaN,NaN,NaN,NaN
6945,용인시,효천마을 신안인스빌 1단지,처인구모현읍,669,2004-12-17,19,5,324,Y,경기도 용인시 처인구모현읍 669,NaN,NaN,NaN,NaN


In [90]:
k_apt_df['lng'] = k_apt_df['lng'].astype(float)
k_apt_df['lat'] = k_apt_df['lat'].astype(float)

In [91]:
k_apt_df = k_apt_df.dropna(axis=0)

In [92]:
k_apt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5995 entries, 0 to 6962
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군명       5995 non-null   object 
 1   공동주택명정보   5995 non-null   object 
 2   읍면동주소     5995 non-null   object 
 3   지번주소      5995 non-null   object 
 4   사용검사일자    5995 non-null   object 
 5   층수        5995 non-null   int64  
 6   동수        5995 non-null   int64  
 7   세대수       5995 non-null   int64  
 8   의무관리대상여부  5995 non-null   object 
 9   addr      5995 non-null   object 
 10  x         5995 non-null   float64
 11  y         5995 non-null   float64
 12  lng       5995 non-null   float64
 13  lat       5995 non-null   float64
dtypes: float64(4), int64(3), object(7)
memory usage: 702.5+ KB


In [93]:
#3. 아파트 위치데이터 리스트 만들기
k_apt_location_data = [[row["lat"], row["lng"]] for _, row in k_apt_df.iterrows()]

In [94]:
path = 'c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/k_bus_info.csv'

bus_line_df = pd.read_csv(path)
bus_line_df.head()

,노선ID,노선명,순번,구간ID,정류소ID,ARS-ID,정류소명,X좌표,Y좌표
0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420


In [95]:

bus_line_df.rename(columns={'경도':'lng','위도':'lat'},inplace = True)

In [96]:
bus_line_df.head()

,노선ID,노선명,순번,구간ID,정류소ID,ARS-ID,정류소명,lng,lat
0,100100124,0017,1,0,102000271,3689,청암자이아파트,126.946517,37.534363
1,100100124,0017,2,102700549,102000204,3298,청암동강변삼성아파트,126.949304,37.533961
2,100100124,0017,3,102700550,102000227,3321,청심경로당,126.950449,37.533744
3,100100124,0017,4,102700551,102000210,3304,원효2동주민센터,126.950904,37.534278
4,100100124,0017,5,102700552,102000212,3306,산천동,126.953984,37.535420


In [97]:
#3. 아파트 위치데이터 리스트 만들기
bus_line_location_data = [[row["lat"], row["lng"]] for _, row in bus_line_df.iterrows()]

In [98]:
#4. 프로젝트(2) ... 3)시각화  ... # Folium 초기 Map 객체 생성 

center = [37.541, 126.986]

map = folium.Map(
    location=center,
    zoom_start= 10
)


In [99]:
# #4. 프로젝트(3)  .. # 2)레이어 적용을 위한 그룹 만들기
# # 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 쏘카 그룹 만들기
socar = plugins.FeatureGroupSubGroup(fg, "socar_zone")
map.add_child(socar)

# 행복주택 그룹 만들기
happy = plugins.FeatureGroupSubGroup(fg, "happy house")
map.add_child(happy)

# 대학교 그룹 만들기
university = plugins.FeatureGroupSubGroup(fg, "university")
map.add_child(university)

# 아파트 그룹 만들기
k_apt = plugins.FeatureGroupSubGroup(fg, "APT")
map.add_child(k_apt)

# 지하철 그룹 만들기
subway = plugins.FeatureGroupSubGroup(fg, "subway")
map.add_child(subway)

# 버스 그룹 만들기
bus = plugins.FeatureGroupSubGroup(fg, "bus")
map.add_child(bus)

# # 경기도 shape
# k_shape = plugins.FeatureGroupSubGroup(fg, "shape")
# map.add_child(k_shape)

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

In [100]:
mg1 = MarkerCluster()
for _, row in k_socar_zone_df.iterrows():
    mg1.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['zone_name'],
               icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")
              )
    )

mg1.add_to(socar)

In [101]:
mg2 = MarkerCluster()
for _, row in happy_house_df.iterrows():
    mg2.add_child(    
        folium.Marker(location = [row['lat'], row['lng']],
               popup=row['단지'],
               icon=folium.Icon(color='red',icon='house', prefix="fa")
              )
    )
    
mg2.add_to(happy)

In [102]:
plugins.HeatMap(k_university_location_data).add_to(university)

In [103]:
plugins.HeatMap(k_apt_location_data).add_to(k_apt)

In [104]:
plugins.HeatMap(k_subway_location_data).add_to(subway)

In [105]:
plugins.HeatMap(bus_line_location_data).add_to(bus)

In [106]:
# mg3 = MarkerCluster()
# for _, row in k_university_df.iterrows():
#     mg3.add_child(    
#         folium.Marker(location = [row['lat'], row['lng']],
#                popup=row['학교명'],
#                icon=folium.Icon(color='green',icon='school',prefix="fa")
#               )
#     )

# mg3.add_to(university)

In [107]:
# mg4 = MarkerCluster()
# for _, row in k_apt_df.iterrows():
#     mg4.add_child(    
#         folium.Marker(location = [row['lat'], row['lng']],
#                popup=row['공동주택명정보'],
#                icon=folium.Icon(color='orange',icon='building',prefix="fa")
#               )
#     )

# mg4.add_to(k_apt)

In [108]:
# mg5 = MarkerCluster()
# for _, row in k_subway_info_df.iterrows():
#     mg5.add_child(    
#         folium.Marker(location = [row['lat'], row['lng']],
#                popup=row['역명'],
#                icon=folium.Icon(color='gray',icon='subway-train',prefix="fa")
#               )
#     )

# mg5.add_to(subway)

In [109]:
# mg6 = MarkerCluster()
# for _, row in bus_line_df.iterrows():
#     mg6.add_child(    
#         folium.Marker(location = [row['lat'], row['lng']],
#                popup=row['노선명'],
#                icon=folium.Icon(color='lightgray',icon='van-shuttle',prefix="fa")
#               )
#     )

# mg6.add_to(bus)

In [110]:
#4 경기 구 data   
k_geo_data_path = 'c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/001_map_data/kyungki_region.json'
k_geo_data = json.load(
    open(k_geo_data_path, encoding="utf-8"))



In [111]:
# folium.GeoJson(
#     k_geo_data,
#     popup="SGG_NM"
#     #name="SGG_NM"
# ).add_to(k_shape)

In [112]:
#4. 프로젝트(3)  ..6) 생성한 map 확인
map


In [113]:
map.save("c:/users/user/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/combine_map_11.html")